# Data Science Capstone Project

### This is the Shareable Jupyter Notebook for the Capstone project.

In [2]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Introduction/Business Problem

The problem at hand is find a solution that can help predict the severity of an accident if an accident were to occur given various environmental information. This is to help potential stakeholders which in this case includes those that have to drive to get to their final destination every day. Given various inputs of a particular day, how can we determine if there will be an accident on the way to a destination? If we can find a method to answer this problem, we can give the stakeholders that insight of choosing a different method of transportation. Not only could this insight potentially offset any risk that the person would incur if they were the ones to get an accident, but it could also allow for their ability to reach their final destination within the time frame that they were expecting to due to not being stuck in traffic waiting for the situation to clear up.


## Data

Using the example dataset’s metadata file, we can see that it is for the relatively high density population within the city of Seattle, Washington. Within this dataset, we can see that it has 37 attributes that can be of interest, including ones that detail the weather, location and level of accidents, the road conditions, among many others. Since we are trying to determine the probability of an accident occurring, and how bad or severe of an accident it will be, we will use the severity attribute as the target variable within our model. In order to decide which attributes would be of most use or importance for the model building, I will apply a feature selection process by utilizing attributes that provide the most information gain on the data, given the target variable. In addition, this dataset also contains an abundance of over 70,000 samples or observations that we can use to train a machine learning model and test against. Since we’ll be working with classification, some classification models that come to mind that may be of use include building a Decision Tree, using K-Nearest Neighbors, Support Vector Machines, and/or Logistic Regression. Once I’ve built each model, I will do a model evaluation based on a couple of scoring mechanism such as the Jaccard Score, Average F1-Score, and LogLoss metrics to see which model performed the best. 

## Methodlogy/Data Exploration

In [19]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
#!conda install -c anaconda seaborn -y
import seaborn as sns
%matplotlib inline

In [20]:
dc = pd.read_csv(r'Data-Collisions.csv')
#Checking Null Values
(dc["ROADCOND"].isnull()==True).sum()
dc.head()

/home/fperez/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0.0,0.0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0.0,0.0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0.0,0.0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0.0,0.0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0.0,0.0,N


In [12]:
dc.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT       float64
PEDCOUNT          float64
PEDCYLCOUNT       float64
VEHCOUNT          float64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE      float64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY        float64
CROSSWALKKEY      float64
HITPARKEDCAR       object
dtype: objec

In [123]:
Features = ['SEVERITYCODE', 'OBJECTID', 'INCDTTM', 'WEATHER','ROADCOND','LIGHTCOND','SPEEDING']
df = dc[Features]


Removing accidents where there is missing times decreased the number of total observations significantly. May need to add it back and just remove using the date and time.

Want to use SeverityCode 0 to remove unknown records, speeding to remove accidents caused by speeding since its not an environemntal cause, Remove observations with missing data on Road Conditions, Light Condtions, and weather conditions.

In [124]:
df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])

#Removing Null Times/Separating Date&Time Column into 2.
print(df.shape)
dtna_con = (df['INCDTTM'].isnull()==True)
df = df.loc[~(dtna_con),:]
print(df.shape)
timena_con =(df['INCDTTM'].apply(lambda x: str(x.time())!= "00:00:00"))
#print(df['INCDTTM'].apply(lambda x: str(x.time())!= "00:00:00"))
df = df.loc[timena_con,:]

df['Date'],df['Time'] = df['INCDTTM'].apply(lambda x:x.date()),df['INCDTTM'].apply(lambda x:x.time())

df.drop('INCDTTM', axis=1, inplace=True)
print(df.shape)

#Now removing accidents caused by speeding specifically
spd_con = (df['SPEEDING'].apply(lambda x:x=='Y'))
df = df.loc[(~spd_con),:]
df.drop('SPEEDING', axis=1, inplace=True)
print(df.shape)

#Removing samples wth no weather data
weather_con = (df['WEATHER'].apply(lambda x: pd.isnull(x)==True\
                    or x=='Unknown'\
                    or x=='Other')\
                    )

df = df.loc[(~weather_con),:]
print(df.shape)

#Removing samples wth no Road Condition data
road_con = (df['ROADCOND'].apply(lambda x: pd.isnull(x)==True\
                    or x=='Unknown'\
                    or x=='Other')\
                    )

df = df.loc[(~road_con),:]
print(df.shape)

#Removing samples wth no Lighting Condition data
light_con = (df['LIGHTCOND'].apply(lambda x: pd.isnull(x)==True\
                    or x=='Unknown'\
                    or x=='Other')\
                    )

df = df.loc[~(light_con),:]
print(df.shape)

/home/fperez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(71646, 7)
(71645, 7)
(50652, 8)
(47809, 7)
(43926, 7)
(43694, 7)
(42961, 7)


In [125]:
print(set(df['SEVERITYCODE'].values))
#road_con = (df['LIGHTCOND'].apply(lambda x: pd.isnull(x)==True\
#                    or x=='Unknown'\
#                    or x=='Other')\
#                    )
#road_con.sum()

#(df['SPEEDING'].apply(lambda x:np.isnan(x)))

{1, 2}


In [152]:
#finding which months have the most accidents
df['Month'] = pd.to_datetime(df['Date']).dt.month.values
df['Month'].head()
df['Dayofweek'] = pd.to_datetime(df['Date']).dt.dayofweek
df.head()

#bins = np.linspace(df.Dayofweek.min(), df.Dayofweek.max(), 10)
#g = sns.FacetGrid(df, col="Month", hue="SEVERITYCODE", palette="Set1", col_wrap=2)
#g.map(plt.hist, 'Dayofweek', bins=bins, ec="k")
#g.axes[-1].legend()
#plt.show()

,SEVERITYCODE,OBJECTID,WEATHER,ROADCOND,LIGHTCOND,Date,Time,Month,Dayofweek
0,2,1,Overcast,Wet,Daylight,2013-03-27,14:54:00,3,2
1,1,2,Raining,Wet,Dark - Street Lights On,2006-12-20,18:55:00,12,2
2,1,3,Overcast,Dry,Daylight,2004-11-18,10:20:00,11,3
3,1,4,Clear,Dry,Daylight,2013-03-29,09:26:00,3,4
4,2,5,Raining,Wet,Daylight,2004-01-28,08:04:00,1,2


In [157]:
Feature2 = pd.concat([df[["Month", "Dayofweek"]],\
                     pd.get_dummies(df['WEATHER']),\
                     pd.get_dummies(df['ROADCOND']),\
                     pd.get_dummies(df['LIGHTCOND'])],\
                     axis=1)
X = Feature2
print(X.shape)
X.head()

(42961, 24)


,Month,Dayofweek,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Overcast,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,...,Snow/Slush,Standing Water,Wet,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk
0,3,2,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,12,2,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,11,3,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,4,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,2,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [159]:
y = df['SEVERITYCODE'].values
y[0:5]

array([2, 1, 1, 1, 2])

## Normalizing Data

In [161]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[-1.03217928, -0.48497591, -0.01930206, -1.3674086 , -0.05225742,
         2.26481899, -0.46260036, -0.01447537, -0.02772596, -0.07058817,
        -1.6673675 , -0.09076069, -0.02103467, -0.0236423 , -0.07542194,
        -0.02158137,  1.73552501, -0.09269605, -0.08172042, -0.6003936 ,
        -0.00482467, -0.11450616,  0.69851979, -0.19000949],
       [ 1.65787509, -0.48497591, -0.01930206, -1.3674086 , -0.05225742,
        -0.44153639,  2.16169307, -0.01447537, -0.02772596, -0.07058817,
        -1.6673675 , -0.09076069, -0.02103467, -0.0236423 , -0.07542194,
        -0.02158137,  1.73552501, -0.09269605, -0.08172042,  1.66557404,
        -0.00482467, -0.11450616, -1.43159866, -0.19000949],
       [ 1.35898016,  0.03508125, -0.01930206, -1.3674086 , -0.05225742,
         2.26481899, -0.46260036, -0.01447537, -0.02772596, -0.07058817,
         0.59974781, -0.09076069, -0.02103467, -0.0236423 , -0.07542194,
        -0.02158137, -0.57619452, -0.09269605, -0.08172042, -0.6003936 ,
  